In [6]:
import argparse
import roypy
import time
import queue
from sample_camera_info import print_camera_info
from roypy_sample_utils import CameraOpener, add_camera_opener_options
#from roypy_platform_utils import PlatformHelper

import numpy as np
import matplotlib.pyplot as plt
import cv2

try:
    import roypycy
except ImportError:
    print("Pico Flexx backend requirements (roypycy) not installed properly")
    raise

class MyListener(roypy.IDepthDataListener):
    def __init__(self, q):
        super(MyListener, self).__init__()
        self.queue = q
        self.Listening = True

    def onNewData(self, data):
        t_time = time.time()
        zvalues = []
        zvalues = roypycy.get_depth_data(data)
        zarray = np.asarray(zvalues)
        p = zarray.reshape (-1, data.width)        
        self.queue.put(p)
        print('store time:', (time.time()-t_time))
            

    def paint (self, data):
        """Called in the main thread, with data containing one of the items that was added to the
        queue in onNewData.
        """
        name = "depth"
        cv2.namedWindow(name, cv2.WINDOW_NORMAL)
        cv2.imshow(name, data)
        cv2.waitKey(1)

def main ():
    parser = argparse.ArgumentParser (usage = __doc__)
    add_camera_opener_options (parser)
    parser.add_argument ("--seconds", type=int, default=15, help="duration to capture data")
    #options = parser.parse_args(args=['--rrf', '0108.rrf','--seconds', '5'])
    options = parser.parse_args(args=['--seconds', '5'])

    opener = CameraOpener (options)
    cam = opener.open_camera ()
    
    Replay = True
    if(Replay):
        cam.setUseCase('MODE_5_35FPS_600')

    print_camera_info (cam)
    print("isConnected", cam.isConnected())
    print("getFrameRate", cam.getFrameRate())
    print("UseCase",cam.getCurrentUseCase())

    # we will use this queue to synchronize the callback with the main
    # thread, as drawing should happen in the main thread
    q = queue.Queue()
    l = MyListener(q)
    cam.registerDataListener(l)
    cam.startCapture()
    # create a loop that will run for a time (default 15 seconds)
    process_event_queue (q, l, options.seconds)
    cam.stopCapture()
    
    cv2.destroyAllWindows()

def process_event_queue (q, painter, seconds):
    # create a loop that will run for the given amount of time
    t_end = time.time() + seconds
    while time.time() < t_end:
        try:
            # try to retrieve an item from the queue.
            # this will block until an item can be retrieved
            # or the timeout of 1 second is hit
            t_time = time.time()
            item = q.get(True, 1)
            print('queue time:', (time.time()-t_time))
        except queue.Empty:
            # this will be thrown when the timeout is hit
            break
        else:
            painter.paint (item)
            print("max ",np.max(item))
            print("min",np.min(item))
            

In [8]:
main()

Number of cameras connected:  1
        Camera information
Type:            PICOFLEXX
Width:           224
Height:          171
Operation modes: 10
    MODE_9_5FPS_2000
    MODE_9_10FPS_1000
    MODE_9_15FPS_700
    MODE_9_25FPS_450
    MODE_5_35FPS_600
    MODE_5_45FPS_500
    MODE_MIXED_30_5
        this operation mode has 2 streams
    MODE_MIXED_50_5
        this operation mode has 2 streams
    Low_Noise_Extended
    Fast_Acquisition
Lens parameters: 9
    ('cx', 118.28559112548828)
    ('cy', 87.74105072021484)
    ('fx', 213.8031768798828)
    ('fy', 213.8031768798828)
    ('k1', 0.4155448377132416)
    ('k2', -4.7316107749938965)
    ('k3', 8.45906925201416)
    ('p1', 7.605663946304829e-16)
    ('p2', 4.939198934392371e-16)
CameraInfo items: 8
    ('BRIDGE_TYPE', 'Enclustra')
    ('MODULE_IDENTIFIER', '00000000')
    ('MODULE_IDENTIFIER_HASH', '558161692')
    ('MODULE_SERIAL', '0')
    ('MODULE_SUFFIX', '')
    ('IMAGER', 'M2450_A12_AIO')
    ('PROCESSING_NAME', 'Spectre')
  

queue time: 0.02686309814453125
max  0.6819958686828613
min 0.0
store time:queue time: 0.0
 0.02498626708984375
max  0.6604923605918884
min 0.0
store time:queue time: 0.0
 0.02593207359313965
max  0.6957786679267883
min 0.0
store time: 0.0
queue time: 0.026836156845092773
max  0.65073561668396
min 0.0
store time: 0.00099945068359375
queue time: 0.023968219757080078
max  0.65073561668396
min 0.0
store time: 0.0
queue time: 0.025960683822631836
max  0.6866471767425537
min 0.0
store time: queue time: 0.025880813598632812
0.0
max  0.7010080218315125
min 0.0
store time: 0.0009999275207519531
queue time: 0.026866436004638672
max  0.6346988081932068
min 0.0
store time: 0.0
queue time: 0.025929927825927734
max  0.6395996809005737
min 0.0
store time: 0.0
queue time: 0.025963544845581055
max  0.6573686599731445
min 0.0
store time:queue time: 0.025885820388793945
 0.0
max  0.6859361529350281
min 0.0
store time:queue time: 0.0
 0.025941133499145508
max  0.6563876867294312
min 0.0
store time:queue 

In [22]:
#!jupyter nbconvert --to script Test.ipynb

[NbConvertApp] Converting notebook Test.ipynb to script
[NbConvertApp] Writing 5855 bytes to Test.py
